In [1]:
'''
Notebook for FIG S?H - comparing the effects of muttating synthetic and native genes
'''
# By Kirill Sechkar

# PACKAGE IMPORTS ------------------------------------------------------------------------------------------------------
import numpy as np
import jax
import jax.numpy as jnp
import functools
from diffrax import diffeqsolve, Dopri5, ODETerm, SaveAt, PIDController, SteadyStateEvent
import pandas as pd
import pickle
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, palettes as bkpalettes, transform as bktransform
from math import pi
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
from contourpy import contour_generator as cgen
import time

# set up jax
from jax.lib import xla_bridge
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
print(xla_bridge.get_backend().platform)

# set up bokeh
bkio.reset_output()
bkio.output_notebook() 

# OWN CODE IMPORTS -----------------------------------------------------------------------------------------------------
import synthetic_circuits as circuits
from cell_model import *
from get_steady_state import *
from Fig2.design_guidance_tools import *

/tmp/ipykernel_7131/2454757870.py:25: DeprecationWarning: jax.lib.xla_bridge.get_backend is deprecated; use jax.extend.backend.get_backend.
  print(xla_bridge.get_backend().platform)
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


cpu


Loading BokehJS ...

In [2]:
 # INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, circuit_eff_m_het_div_k_het,  \
    par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, _ = cellmodel_auxil.add_circuit(
    circuits.punisher_b_initialise,
    circuits.punisher_b_ode,
    circuits.punisher_b_F_calc,
    circuits.punisher_sep_b_eff_m_het_div_k_het,
    par, init_conds)  # load the circuit

In [3]:
# SPECIFY THE CIRCUIT'S DEFAULT PARAMETERS

# BURDENSOME SYNTHETIC GENE
par['c_b'] = 1
par['a_b'] = 1e5

# PUNISHER
# switch gene conc
par['c_switch'] = 10.0  # gene concentration (nM)
par['a_switch'] = 400.0  # promoter strength (unitless)
par['d_switch']=0.01836
# integrase - expressed from the switch gene's operon, not its own gene => c_int, a_int irrelevant
par['k+_int'] = par['k+_switch']/80.0  # RBS weaker than for the switch gene
par['d_int'] = 0.0#0.01836 # rate of integrase degradation per protease molecule (1/nM/h)
# CAT (antibiotic resistance) gene
init_conds['cat_pb'] = 10.0  # gene concentration (nM) - INITIAL CONDITION< NOT PARAMETER as it can be cut out by the integrase
par['a_cat'] = 500.0  # promoter strength (unitless)
# synthetic protease gene
par['c_prot'] = 10.0  # gene concentration (nM)
par['a_prot'] = 25.0  # promoter strength (unitless)
init_conds['p_prot'] = 1500.0 # if zero at start, the punisher's triggered prematurely

# punisher's transcription regulation function
par['K_switch'] = 300.0  # Half-saturation constant for the self-activating switch gene promoter (nM)
par['eta_switch'] = 2 # Hill coefficient for the self-activating switch gene promoter (unitless)
par['baseline_switch'] = 0.025  # Baseline value of the switch gene's transcription activation function
par['baseline_switch_alt'] = 0
par['p_switch_ac_frac'] = 0.85  # active fraction of protein (i.e. share of molecules bound by the inducer)

# CULTURE MEDIUM
init_conds['s'] = 0.5
par['h_ext'] = 10.5 * (10.0 ** 3)

In [4]:
# DETERMINISTIC SIMULATION PARAMETERS (FOR FINDING GROWTH RATES)
savetimestep = 1  # save time step
rtol = 1e-6  # relative tolerance for the ODE solver
atol = 1e-6  # absolute tolerance for the ODE solver
# simulation time frames
tf = (0,50) # time frame for simulation before burdensome gene loss

In [5]:
# DEFINE THE COMBINATIONS OF BURDENSOME GENE EXPRESSION AND CULTURE MEDIUM QUALITIES

first_a_b_change = -par['a_b']*0.75
first_s_change = 0.08
second_a_b_change = -par['a_b']*0.2
second_s_change = 0.03

mutcombs={
    'original': {'a_b': par['a_b'], 's': init_conds['s']},    # original engineered cell
    'synth mut': {'a_b': par['a_b']+first_a_b_change, 's': init_conds['s']},    # burdensome gene mutated
    'nat mut': {'a_b': par['a_b'], 's': init_conds['s']+first_s_change},    # culture medium quality improved
    'synth mut, then nat mut': {'a_b': par['a_b']+first_a_b_change, 's': init_conds['s']+second_s_change},    # burdensome gene mutated, then culture medium quality improved
    'synth mut, then synth mut': {'a_b': par['a_b']+first_a_b_change+second_a_b_change, 's': init_conds['s']},    # burdensome gene mutated twice
    'nat mut, then synth mut': {'a_b': par['a_b']+second_a_b_change, 's': init_conds['s']+first_s_change},    # culture medium quality improved, then burdensome gene mutated
    'nat mut, then nat mut': {'a_b': par['a_b'], 's': init_conds['s']+first_s_change+second_s_change}    # culture medium quality improved twice
          }

In [6]:
# FOR EACH COMBINATION OF BURDENSOME GENE EXPRESSION AND CULTURE MEDIUM QUALITY, FIND THE STEADY-STATE GROWTH RATE AND BURDEN MARGIN

DXis = {}
l_ss = {}
for mutcomb in mutcombs.keys():
    cur_par = par.copy() # current parameter dictionary
    cur_par['a_b'] = mutcombs[mutcomb]['a_b'] # set the burdensome gene promoter strength
    cur_par['k_sxf']=0.0    # not considering punisher action explicitly
    cur_inits = init_conds.copy() # current initial conditions
    cur_inits['s'] = mutcombs[mutcomb]['s'] # set the nutrient quality
    
    # get the cellular variables in steady state without burden
    e, F_r, h, xis, chis = values_for_analytical(cur_par, ode_with_circuit, cur_inits,
                                                 circuit_genes, circuit_miscs,
                                                 circuit_name2pos,
                                                 circuit_F_calc,
                                                 circuit_eff_m_het_div_k_het)
    # record the cellular variables
    cellvars = {'e': e, 'F_r': F_r, # translation elongation rate and ribosome trnscription regulation
                'h': h, # intracellular chlorampenicol concentration
                # burden values
                'xi_a': xis['a'], 'xi_r': xis['r'], 'xi_other_genes': xis['other'], 'xi_cat': xis['cat'],
                'xi_switch_max': xis['switch (max)'], 'xi_int_max': xis['int (max)'], 'xi_prot': xis['prot'],
                # protein degradation correction factors for the switch protein and the integrase
                'chi_switch': chis['switch'], 'chi_int': chis['int']}
    # the burden value xi with the synthetic gene
    xi = xis['a'] + xis['r'] + xis['cat'] + xis['prot'] + xis['other']
    
    # get the threshold burden value for the punisher's switching
    _, xi_threshold, _, _, _ = threshold_mfchanges(cur_par, cellvars)
    
    # record the burden margin
    DXis[mutcomb] = np.float64(xi - xi_threshold)
    
    # get the steady-state growth rate
    sol = ode_sim(cur_par,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(cur_inits, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(cur_par, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
    ts = np.array(sol.ts)
    xs = np.array(sol.ys)
    _, ls, _, _, _, _, _, _ = cellmodel_auxil.get_e_l_Fr_nu_psi_T_D_Dnodeg(ts, xs, cur_par, circuit_genes, circuit_miscs, circuit_name2pos,
                                                                           circuit_eff_m_het_div_k_het)
    # record the growth rate
    l_ss[mutcomb]=np.float64(ls[-1])

# get the grwoth benefits
Dls = {}
for mutcomb in mutcombs.keys():
    Dls[mutcomb] = l_ss[mutcomb]/l_ss['original']-1

In [7]:
# PLOT

# set ranges for the plot
gb_plot_range=(0, 0.2)
Dxi_plot_range=(-5000, 20000)

# calculate label poisitions
nat_mut_label_pos = ((Dls['original'] + Dls['nat mut']) / 2,
                     (DXis['original'] + DXis['nat mut']) / 2)
nat_mut_further_mut_label_pos = ((Dls['nat mut'] + Dls['nat mut, then nat mut']) / 2,
                                 (DXis['nat mut'] + DXis['nat mut, then nat mut']) / 2)
synth_mut_label_pos = ((Dls['original'] + Dls['synth mut']) / 2,
                       (DXis['original'] + DXis['synth mut']) / 2)
synth_mut_further_mut_label_pos = ((Dls['synth mut'] + Dls['synth mut, then synth mut']) / 2,
                                   (DXis['synth mut'] + DXis['synth mut, then synth mut']) / 2)

# set up the figure
gbdxi_fig = bkplot.figure(
    frame_width=240,
    frame_height=180,
    x_axis_label="Δλ (relative growth benefit)",
    y_axis_label="ΔΞ (burden margin)",
    x_range=gb_plot_range,
    y_range=Dxi_plot_range,
    tools="box_zoom,pan,hover,reset,save"
)
gbdxi_fig.xaxis.axis_label_text_font_size = '9pt'
gbdxi_fig.yaxis.axis_label_text_font_size = '9pt'
gbdxi_fig.output_backend = 'svg'
# add shading for where punishment is triggered
gbdxi_fig.add_layout(
    bkmodels.PolyAnnotation(xs=[gb_plot_range[0], gb_plot_range[0], gb_plot_range[1], gb_plot_range[1]],
                            ys=[Dxi_plot_range[0], 0, 0, Dxi_plot_range[0]],
                            fill_color='gray', fill_alpha=0.5, hatch_pattern='left_diagonal_dash',
                            line_color=None)
)
gbdxi_fig.add_layout(bkmodels.Label(x=gb_plot_range[0], y=Dxi_plot_range[0],
                                x_offset=8, y_offset=8,
                                text='Punisher triggered',
                                text_font_size='10pt',
                                background_fill_color='white', background_fill_alpha=1))
# define the arrow heads
native_head = bkmodels.VeeHead(size=7.5, fill_color='blue', line_color='blue')
synth_head = bkmodels.VeeHead(size=7.5, fill_color='red', line_color='red')
# plot the native gene mutation and mutations which might follow
gbdxi_fig.add_layout(bkmodels.Arrow(x_start=Dls['original'], y_start=DXis['original'], 
                                    x_end=Dls['nat mut'], y_end=DXis['nat mut'],
                                    end=native_head, line_color='blue', 
                                    line_width=1.5, line_dash='solid')) # first native gene mutation
gbdxi_fig.add_layout(bkmodels.Label(x=nat_mut_label_pos[0], y=nat_mut_label_pos[1],
                                x_offset=3, y_offset=0,
                                text='Native gene mutation',
                                text_font_size='8pt',
                                text_color='blue',
                                background_fill_alpha=0))
gbdxi_fig.add_layout(bkmodels.Arrow(x_start=Dls['nat mut'], y_start=DXis['nat mut'],
                                    x_end=Dls['nat mut, then nat mut'], y_end=DXis['nat mut, then nat mut'],
                                    end=native_head, line_color='blue', 
                                    line_width=1.5, line_dash='dotted')) # second native gene mutation
gbdxi_fig.add_layout(bkmodels.Arrow(x_start=Dls['nat mut'], y_start=DXis['nat mut'],
                                    x_end=Dls['nat mut, then synth mut'], y_end=DXis['nat mut, then synth mut'],
                                    end=synth_head, line_color='red', 
                                    line_width=1.5, line_dash='dotted')) # second synthetic gene mutation following the first native gene mutation
gbdxi_fig.add_layout(bkmodels.Label(x=nat_mut_further_mut_label_pos[0], y=nat_mut_further_mut_label_pos[1],
                                x_offset=3, y_offset=0,
                                text='Further mutations',
                                text_font_size='8pt',
                                background_fill_alpha=0))
# plot the synthetic gene mutation and mutations which might follow
gbdxi_fig.add_layout(bkmodels.Arrow(x_start=Dls['original'], y_start=DXis['original'], 
                                    x_end=Dls['synth mut'], y_end=DXis['synth mut'],
                                    end=synth_head, line_color='red', 
                                    line_width=1.5, line_dash='solid')) # first synthetic gene mutation
gbdxi_fig.add_layout(bkmodels.Label(x=synth_mut_label_pos[0], y=synth_mut_label_pos[1],
                                x_offset=-3, y_offset=-24,
                                text_align='right',
                                text='Synth. gene\nmutation',
                                text_font_size='8pt',
                                text_color='red',
                                background_fill_alpha=0))
gbdxi_fig.add_layout(bkmodels.Arrow(x_start=Dls['synth mut'], y_start=DXis['synth mut'],
                                    x_end=Dls['synth mut, then nat mut'], y_end=DXis['synth mut, then nat mut'],
                                    end=native_head, line_color='blue', 
                                    line_width=1.5, line_dash='dotted')) # second native gene mutation following the first synthetic gene mutation
gbdxi_fig.add_layout(bkmodels.Arrow(x_start=Dls['synth mut'], y_start=DXis['synth mut'],
                                    x_end=Dls['synth mut, then synth mut'], y_end=DXis['synth mut, then synth mut'],
                                    end=synth_head, line_color='red', 
                                    line_width=1.5, line_dash='dotted')) # second synthetic gene mutation
gbdxi_fig.add_layout(bkmodels.Label(x=synth_mut_further_mut_label_pos[0], y=synth_mut_further_mut_label_pos[1],
                                x_offset=-16, y_offset=3,
                                text_align='right',
                                text='Further mutations',
                                text_font_size='8pt',
                                background_fill_alpha=0))

# set up the legend
# gbdxi_fig.legend.location = 'top_right'
# gbdxi_fig.legend.label_text_font_size = '8pt'
# gbdxi_fig.legend.click_policy = 'hide'
# gbdxi_fig.legend.title = 'Mutation:'
# gbdxi_fig.legend.title_text_font_size = '8pt'
# gbdxi_fig.legend.margin = 5
# gbdxi_fig.legend.spacing = 5
# gbdxi_fig.legend.padding = 5

# show the plot
bkplot.show(gbdxi_fig)